In [126]:
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import cv2
from PIL import Image
import os

In [127]:
# load the saved model
model = load_model('my_model.h5')

In [128]:
N = 200
x_test = np.zeros((N, 112, 112, 1), 'float')
y_test = np.zeros((N), 'float')

count = 0
class_labels = sorted(os.listdir("Dataset/test"))
for i, class_label in enumerate(class_labels):
    image_files = sorted(os.listdir(os.path.join("Dataset/test", class_label)))
    for image_file in image_files:
        im = cv2.imread(os.path.join("Dataset/test", class_label, image_file), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (112, 112))
        x_test[count, :, :, 0] = im / 255.
        y_test[count] = i
        count += 1

y_test = to_categorical(y_test)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_test (200, 112, 112, 1)
y_test (200, 4)


In [129]:
# use the model to make predictions
predictions = model.predict(x_test)

# Get the class predictions and confidence scores
class_preds = predictions.argmax(axis=1)
confidences = np.max(predictions, axis=-1)

# Define the class names
class_names = ['B', 'D', 'R', 'S']

# Define the test data directory
test_dir = 'Dataset/test/'

test_images = []
for class_folder in os.listdir(test_dir):
    class_folder_path = os.path.join(test_dir, class_folder)
    if os.path.isdir(class_folder_path):
        for image_file in os.listdir(class_folder_path):
            if image_file.endswith('.jpg'):
                image_path = os.path.join(class_folder_path, image_file)
                test_images.append(image_path)

7/7 [==============================] - 0s 42ms/step


In [130]:
# Save the predictions to a file
with open('result.txt', 'w') as f:
    for i in range(len(test_images)):
        image_name = os.path.basename(test_images[i])
        class_name = class_names[class_preds[i]]
        confidence = confidences[i]
        f.write("{}: ,{}\n".format(
            image_name, class_name))

In [131]:
# evaluate the model on test data
score = model.evaluate(x_test, y_test)

# print the model's accuracy
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 42ms/step - loss: 0.3421 - accuracy: 0.9350
Test accuracy: 0.9350000023841858


In [132]:
import tensorflow as tf

model = tf.keras.models.load_model('my_model.h5')

model.summary()



Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 112, 112, 1)]     0         
                                                                 
 conv2d_19 (Conv2D)          (None, 110, 110, 32)      320       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 55, 55, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 53, 53, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 26, 26, 64)       0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 43264)             0   